In [1]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD= os.getenv('POSTGRES_PASSWORD')

In [2]:
nba= pd.read_csv('nba.csv', low_memory= False)
nba

,Unnamed: 0,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,0,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,1,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,2,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,3,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,4,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112118,112118,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7.0,...,0.107914,13.08,33.110667,19.232562,0.0,2.0,77.0,21.0,0.0,57.207786
112119,112119,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7.0,...,0.036079,6.00,25.470833,20.228571,5.0,45.0,43.0,7.0,0.0,58.202391
112120,112120,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7.0,...,0.150943,4.00,24.083333,13.228788,0.0,0.0,0.0,9.0,91.0,49.630640
112121,112121,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7.0,...,0.094340,12.64,34.783333,27.691667,0.0,44.0,48.0,8.0,0.0,58.923515


# Original Data

In [3]:
pd.set_option('display.max_rows', 50)
nba.head(30).T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
game_id,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,...,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170CHO,202202170CHO,202202170CHO,202202170CHO
game_date,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,...,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,2,2
H_A,A,A,A,A,A,A,A,A,A,A,...,H,H,H,H,H,H,A,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SG%,36.0,0.0,32.0,10.0,0.0,7.0,0.0,0.0,0.0,0.0,...,82.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,0.0
SF%,60.0,4.0,67.0,0.0,0.0,62.0,0.0,33.0,16.0,0.0,...,3.0,0.0,9.0,0.0,27.0,0.0,68.0,0.0,36.0,0.0
PF%,4.0,85.0,0.0,0.0,0.0,31.0,0.0,57.0,70.0,0.0,...,0.0,0.0,51.0,0.0,67.0,4.0,32.0,0.0,1.0,94.0
C%,0.0,11.0,0.0,0.0,100.0,0.0,0.0,10.0,14.0,100.0,...,0.0,0.0,40.0,100.0,5.0,96.0,1.0,0.0,0.0,6.0


In [4]:
##Entity- table
##Attribute- column
##Primary Key- one or more columns in a table that are unique on every row


## First Normal Form

In [5]:
nba1= nba.drop(['Inactives'], axis=1)
#nba1.head(3).T

In [6]:
##primary dependent depends on stuffs outside the primary key
##The key(First normal form), the whole key (Second normal form), nothing but the key (Third normal form).

In [7]:
##Primay key for this data is game id and player id

# Second Normal Form

In [8]:
players = nba1[['player_id', 'player']].drop_duplicates()
#players

In [9]:
nba2 = nba1.drop(['player'], axis=1) 

In [10]:
games = nba1[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()
#games

In [11]:
nba = nba1.drop(['game_date', 'OT', 'season'], axis=1)
#nba.head(3).T

# Third Normal Form

In [12]:
nba.columns

Index(['Unnamed: 0', 'game_id', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score',
       'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct',
       'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg', 'player',
       'player_id', 'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'plus_minus', 'did_not_play', 'is_inactive',
       'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct',
       'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct',
       'usg_pct', 'off_rtg', 'def_rtg', 'bpm', 'minutes', 'double_double',
       'triple_double', 'DKP', 'FDP', 'SDP', 'DKP_per_minute',
       'FDP_per_minute', 'SDP_per_minute', 'pf_per_minute', 'ts',
       'last_60_minutes_per_game_starting', 'last_60_minutes_p

In [13]:


# Create games dataframe
games = nba1[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()

# Create team_game dataframe
team_game = nba[['game_id', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates()

# Display transposed first 4 rows of team_game
team_game_display = team_game.head(4).T
team_game_display

# Attempt to recreate the team_game dataframe
# 

,0,13,26,37
game_id,202202170BRK,202202170BRK,202202170CHO,202202170CHO
Team_Abbrev,WAS,BRK,MIA,CHO
Team_Score,117,103,111,107
Team_pace,94.5,94.5,88.8,88.8
Team_efg_pct,0.627,0.483,0.471,0.453
Team_tov_pct,13.5,13.1,11.1,13.6
Team_orb_pct,22.9,33.3,26.8,28.1
Team_ft_rate,0.157,0.191,0.147,0.221
Team_off_rtg,123.8,109.0,103.4,99.7
Opponent_Abbrev,BRK,WAS,CHO,MIA


In [14]:
import pandas as pd


# Create games dataframe
games = nba1[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()

# Create team_game dataframe
team_game = nba[['game_id', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates()

# Display transposed first 4 rows of team_game
team_game_display = team_game.head(4).T
team_game_display


,0,13,26,37
game_id,202202170BRK,202202170BRK,202202170CHO,202202170CHO
Team_Abbrev,WAS,BRK,MIA,CHO
Team_Score,117,103,111,107
Team_pace,94.5,94.5,88.8,88.8
Team_efg_pct,0.627,0.483,0.471,0.453
Team_tov_pct,13.5,13.1,11.1,13.6
Team_orb_pct,22.9,33.3,26.8,28.1
Team_ft_rate,0.157,0.191,0.147,0.221
Team_off_rtg,123.8,109.0,103.4,99.7
Opponent_Abbrev,BRK,WAS,CHO,MIA


In [15]:
#team_game.head(4).T

In [16]:
 pd.set_option('display.max_columns', 80)
 
nba.query("game_id == '202202170'")


,Unnamed: 0,game_id,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev,Opponent_Score,Opponent_pace,Opponent_efg_pct,Opponent_tov_pct,Opponent_orb_pct,Opponent_ft_rate,Opponent_off_rtg,player,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,did_not_play,is_inactive,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,bpm,minutes,double_double,triple_double,DKP,FDP,SDP,DKP_per_minute,FDP_per_minute,SDP_per_minute,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes


In [17]:
player_game = nba.drop(['H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg'], axis=1)

#'Inactives'
# Display the first few rows of the player_game dataframe as a check
#player_game.head()

In [18]:
dbserver = psycopg2.connect(
     user = 'postgres',
     password = POSTGRES_PASSWORD,
     host = 'postgres',
     #host = 'host.docker.internal',
     port= '5432'   
)
dbserver.autocommit = True

In [19]:
 cursor = dbserver.cursor()

In [20]:
try:
    cursor.execute('CREATE DATABASE nba')
except:
    cursor.execute('DROP DATABASE nba')
    cursor.execute('CREATE DATABASE nba')

In [21]:
engine = create_engine("postgresql+psycopg2://{user}:{pw}@{service}/{db}".format (
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    db = 'nba'
))

In [22]:
games.columns = [x.lower() for x in games.columns]
players.columns = [x.lower() for x in players.columns]
team_game.columns = [x.lower() for x in team_game.columns]
player_game.columns = [x.lower() for x in player_game.columns]

In [23]:
games.to_sql('games', con = engine, index=False,  chunksize =1000, if_exists= 'replace')

3197

In [24]:
players.to_sql('players', con = engine, index=False,  chunksize =1000, if_exists= 'replace')

812

In [25]:
team_game.to_sql('team_game', con = engine, index=False,  chunksize =1000, if_exists= 'replace')

6394

In [26]:
player_game.to_sql('player_game', con = engine, index=False,  chunksize =500, if_exists= 'replace')

112123

In [27]:
myquery = '''

SELECT *
FROM games

'''
pd.read_sql_query(myquery, con= engine)


,game_id,game_date,ot,season
0,202202170BRK,2022-02-17,0,2022
1,202202170CHO,2022-02-17,2,2022
2,202202170LAC,2022-02-17,0,2022
3,202202170MIL,2022-02-17,0,2022
4,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
3192,202001080GSW,2020-01-08,0,2020
3193,202008020HOU,2020-08-02,0,2020
3194,201911060HOU,2019-11-06,0,2020
3195,201912250GSW,2019-12-25,0,2020


In [31]:
myquery = '''
SELECT *
FROM games
#WHERE season = 2022
WHERE game_date= '2021-04-12
'''
pd.read_sql_query(myquery, con= engine)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "#"
LINE 4: #WHERE season = 2022
        ^

[SQL: 
SELECT *
FROM games
#WHERE season = 2022
WHERE game_date= '2021-04-12
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
#all games where a player scores > 40, and team 120, and calculate % of team

In [ ]:
# myquery = '''
# SELECT game_id, player_id, pg.pts, tg.team_score
# FROM player_game pg
# INNER JOIN team_game tg
#     ON pg.game_id = tg.game_id
# WHERE pts > 40 
# '''

# pd.read_sql_query(myquery, con= engine)

In [29]:
myquery = '''

SELECT tg.game_id, tg.team_abbrev, tg.team_score, tg.opponent_abbrev,g.game_date, g.season
FROM team_game tg

INNER JOIN games g
    ON tg.game_id=g.game_id
WHERE team_abbrev='CLE'
ORDER BY team_score DESC 
LIMIT 10
'''
pd.read_sql_query(myquery, con= engine)

,game_id,team_abbrev,team_score,opponent_abbrev,game_date,season
0,202101200CLE,CLE,147,BRK,2021-01-20,2021
1,202112260CLE,CLE,144,TOR,2021-12-26,2022
2,202002030CLE,CLE,134,NYK,2020-02-03,2020
3,202003080CLE,CLE,132,SAS,2020-03-08,2020
4,202104080OKC,CLE,129,OKC,2021-04-08,2021
5,202012260DET,CLE,128,DET,2020-12-26,2021
6,202002120CLE,CLE,127,ATL,2020-02-12,2020
7,202111070NYK,CLE,126,NYK,2021-11-07,2022
8,202104050SAS,CLE,125,SAS,2021-04-05,2021
9,202101220CLE,CLE,125,BRK,2021-01-22,2021


In [30]:
player_game.dtypes

unnamed: 0                   int64
game_id                     object
player                      object
player_id                   object
starter                      int64
                            ...   
sg%                        float64
sf%                        float64
pf%                        float64
c%                         float64
active_position_minutes    float64
Length: 61, dtype: object

In [32]:
team_game.dtypes

game_id             object
team_abbrev         object
team_score           int64
team_pace          float64
team_efg_pct       float64
team_tov_pct       float64
team_orb_pct       float64
team_ft_rate       float64
team_off_rtg       float64
opponent_abbrev     object
dtype: object